In [ ]:
# Pre-download models (so you can see the progress)
from huggingface_hub import snapshot_download

print("📥 Downloading Echo TTS model (~2GB)...")
snapshot_download("jordand/echo-tts-base", local_dir_use_symlinks=False)
print("✅ Downloaded echo-tts-base")

print("📥 Downloading audio decoder...")
snapshot_download("jordand/fish-s1-dac-min", local_dir_use_symlinks=False)
print("✅ Downloaded fish-s1-dac-min")

print("\n✅ All models downloaded!")
